In [4]:
import pandas as pd
import numpy as np

In [3]:
resident = pd.read_csv('C:/Users/bhg/Desktop/2022빅콘/2022빅콘테스트_데이터분석리그_데이터분석분야_챔피언부문_데이터셋_220908/1.ev_app_resident.csv')
activity = pd.read_csv('C:/Users/bhg/Desktop/2022빅콘/2022빅콘테스트_데이터분석리그_데이터분석분야_챔피언부문_데이터셋_220908/2.ev_app_activity.csv')
activity_resident = pd.read_csv('C:/Users/bhg/Desktop/2022빅콘/2022빅콘테스트_데이터분석리그_데이터분석분야_챔피언부문_데이터셋_220908/3.ev_app_activity_resident.csv')

In [4]:
resident.drop(['ccw_cd','adng_cd'],axis=1,inplace=True)
activity.drop(['ccw_cd','adng_cd'],axis=1,inplace=True)
activity_resident.drop(['ccw_cd','adng_cd','mega_cd','mega_nm'],axis=1,inplace=True)

### 격자에 따라 세 데이터 merge

In [5]:
merged1 = pd.merge(resident,activity,how='outer',on=['cell_id','ccw_nm','adng_nm','cell_xcrd','cell_ycrd','base_dt','time_zone','dow']) 
# 같은 격자내 같은 날짜, 시간대는 묶기

In [6]:
merged2 = pd.merge(merged1,activity_resident, how='outer',on=['cell_id','ccw_nm','adng_nm','cell_xcrd','cell_ycrd','base_dt','time_zone','dow'])
merged2_y = merged2.loc[merged2['ccw_nm']=='용인시'].copy()
merged2_y

,base_dt,dow,ccw_nm,adng_nm,cell_id,cell_xcrd,cell_ycrd,gender_x,age_x,app_web_x,time_zone,count_cust_x,gender_y,age_y,app_web_y,count_cust_y,gender,age,app_web,count_cust
0,20220605,7,용인시,동백1동,86135034,127.15225,37.286300,MALE,6.0,EV라운지,5,3.522,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20220625,6,용인시,상현2동,84375196,127.09008,37.309536,MALE,8.0,파워큐브차저,2,4.231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,20220625,6,용인시,상현2동,84375196,127.09008,37.309536,MALE,8.0,하이차저,2,8.462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,20220614,2,용인시,포곡읍,87958893,127.21660,37.274303,FEMALE,14.0,EV라운지,1,18.440,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,20220619,7,용인시,신봉동,84119243,127.08097,37.323480,MALE,13.0,EV라운지,2,4.335,MALE,13.0,EV라운지,4.335,MALE,13.0,EV라운지,4.335
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
630223,20220620,1,용인시,양지면,90182654,127.29511,37.229393,NaN,NaN,NaN,2,NaN,FEMALE,4.0,EV라운지,3.274,FEMALE,4.0,EV라운지,3.274
630224,20220608,3,용인시,남사읍,85878686,127.14400,37.122220,NaN,NaN,NaN,3,NaN,MALE,9.0,EV라운지,4.140,NaN,NaN,NaN,NaN
630225,20220628,2,용인시,성복동,83687264,127.06575,37.320717,NaN,NaN,NaN,2,NaN,FEMALE,8.0,EV라운지,5.266,NaN,NaN,NaN,NaN
630226,20220628,2,용인시,신봉동,84103247,127.08040,37.324825,NaN,NaN,NaN,5,NaN,MALE,3.0,EV라운지,21.564,NaN,NaN,NaN,NaN


In [7]:
merged2_y.to_csv('bigcon_merged.csv',encoding='cp949')

In [8]:
merged_grids = merged2_y[['adng_nm','cell_id','cell_xcrd','cell_ycrd']].drop_duplicates()

In [9]:
merged_grids.to_csv('merged_grids.csv',encoding='cp949')

#### 격자별 count_cust 합 

In [10]:
grid_countcust = merged2_y.groupby(['adng_nm','cell_id','cell_xcrd','cell_ycrd']).sum()

In [11]:
grid_countcust.reset_index(inplace=True)
grid_countcust['tot_count_cust'] = grid_countcust['count_cust_x'] + grid_countcust['count_cust_y']+ grid_countcust['count_cust']
grid_countcust = grid_countcust[['adng_nm','cell_id','cell_xcrd','cell_ycrd','tot_count_cust']]

In [12]:
merged2_y.loc[merged2_y['adng_nm']=='역삼동'][['cell_id','cell_xcrd', 'cell_ycrd',]].to_csv('역삼동.csv')
# 역삼동은 역북동 삼가동으로 분리 되어 따로 할당 

In [13]:
# 역삼동은 역북동, 삼가동으로 분리되어 재할당
ysd = pd.read_csv("C:/Users/bhg/Desktop/2022빅콘/역삼동좌표.csv")
ysd = ysd[['cell_id','cell_xcrd','cell_ycrd','EMD_KOR_NM']]
ysd = ysd.drop_duplicates()

In [14]:
grid_countcust = pd.merge(ysd,grid_countcust,on=('cell_id','cell_xcrd','cell_ycrd'),how='right')
grid_countcust['읍면동_격자'] = np.where(pd.notnull(grid_countcust['EMD_KOR_NM'])==True, grid_countcust['EMD_KOR_NM'],grid_countcust['adng_nm'])
grid_countcust = grid_countcust[['cell_id','cell_xcrd','cell_ycrd','읍면동_격자','tot_count_cust']]

In [15]:
# 김량장동으로 찍히는 격자는 중앙동으로 매핑 
grid_countcust = grid_countcust.replace('김량장동','중앙동')
grid_countcust

,cell_id,cell_xcrd,cell_ycrd,읍면동_격자,tot_count_cust
0,84919076,127.109460,37.270844,구갈동,923.508
1,84935076,127.110020,37.271297,구갈동,3.048
2,84935081,127.110010,37.273550,구갈동,1262.039
3,84935082,127.110010,37.274000,구갈동,275.330
4,84935083,127.110010,37.274452,구갈동,181.521
...,...,...,...,...,...
18542,84647202,127.099620,37.319935,풍덕천2동,157.781
18543,84663200,127.100180,37.319490,풍덕천2동,2775.626
18544,84679200,127.100746,37.319940,풍덕천2동,8.130
18545,84695200,127.101300,37.320393,풍덕천2동,10.038


In [16]:
grid_countcust['cell_id'].nunique()

18547

#### 격자, 차량 결합

In [17]:
cars = pd.read_csv("C:/Users/bhg/Desktop/2022빅콘/조사 데이터/자동차등록현황(2022년 9월).csv",encoding='cp949')
cars.rename(columns={'구  분':'동리명_차량','합  계':'합계_차량'},inplace=True)
cars

,동리명_차량,합계_차량,승 용,승 합,화 물,특 수,관 용,자가용,영업용
0,포곡읍,18048,14071,739,3110,128,10,17388,650
1,모현읍,16486,12192,720,3507,67,0,15616,870
2,이동읍,13126,9567,385,3129,45,11,12158,957
3,남사읍,13772,11156,352,2208,56,28,13490,254
4,원삼면,5862,3973,146,1704,39,2,5354,506
5,백암면,6491,3607,255,2467,162,1,5225,1265
6,양지면,12941,9713,409,2699,120,0,11940,1001
7,김량장동,9422,7930,358,1098,36,24,9094,304
8,남동,3792,2584,763,435,10,0,3062,730
9,역북동,14184,12574,338,1208,64,85,13698,401


#####  법정동, 행정동 매핑

In [18]:
dongs = pd.read_csv("C:/Users/bhg/Desktop/2022빅콘/조사 데이터/법정동행정동매핑.csv",encoding='cp949')
dongs.rename(columns={'읍면동명':'읍면동_매핑'},inplace=True)
dongs

,시군구명,읍면동_매핑,동리명,Unnamed: 3
0,용인시 처인구,포곡읍,포곡읍,NaN
1,용인시 처인구,포곡읍,삼계리,NaN
2,용인시 처인구,포곡읍,금어리,NaN
3,용인시 처인구,포곡읍,둔전리,NaN
4,용인시 처인구,포곡읍,영문리,NaN
...,...,...,...,...
117,용인시 수지구,동천동,고기동,NaN
118,용인시 수지구,상현1동,상현동,NaN
119,용인시 수지구,상현2동,상현동,NaN
120,용인시 수지구,상현3동,상현동,NaN


In [19]:
mapping_cars= pd.merge(dongs, cars,how='outer',left_on='동리명',right_on='동리명_차량')
# mapping_dong[['읍면동_매핑','동리명','동리명_차량']]
mapping_cars.dropna(subset=['동리명_차량'],inplace=True)
mapping_cars.drop(['시군구명','동리명','Unnamed: 3'],axis=1,inplace=True)
mapping_cars

,읍면동_매핑,동리명_차량,합계_차량,승 용,승 합,화 물,특 수,관 용,자가용,영업용
0,포곡읍,포곡읍,18048.0,14071.0,739.0,3110.0,128.0,10.0,17388.0,650.0
10,모현읍,모현읍,16486.0,12192.0,720.0,3507.0,67.0,0.0,15616.0,870.0
19,이동읍,이동읍,13126.0,9567.0,385.0,3129.0,45.0,11.0,12158.0,957.0
29,남사읍,남사읍,13772.0,11156.0,352.0,2208.0,56.0,28.0,13490.0,254.0
41,원삼면,원삼면,5862.0,3973.0,146.0,1704.0,39.0,2.0,5354.0,506.0
54,백암면,백암면,6491.0,3607.0,255.0,2467.0,162.0,1.0,5225.0,1265.0
68,양지면,양지면,12941.0,9713.0,409.0,2699.0,120.0,0.0,11940.0,1001.0
79,중앙동,김량장동,9422.0,7930.0,358.0,1098.0,36.0,24.0,9094.0,304.0
80,중앙동,남동,3792.0,2584.0,763.0,435.0,10.0,0.0,3062.0,730.0
81,역북동,역북동,14184.0,12574.0,338.0,1208.0,64.0,85.0,13698.0,401.0


In [20]:
sep_dong = ['동백1동','동백2동','동백3동','풍덕천1동','풍덕천2동','상현1동','상현2동','상현3동','죽전1동','죽전2동','죽전3동','영덕1동','영덕2동']
cal_cars = mapping_cars.loc[~mapping_cars['읍면동_매핑'].isin(sep_dong)]
# mapping_total.groupby('읍면동_인구')['합계_차량', '승  용', '승  합', '화  물', '특  수', '관  용', '자가용', '영업용',].sum()
cal_cars = cal_cars.groupby('읍면동_매핑')['합계_차량', '승  용', '승  합', '화  물', '특  수', '관  용', '자가용', '영업용',].sum()
cal_cars.reset_index(inplace=True)

C:\Users\bhg\AppData\Local\Temp\ipykernel_7280\371494850.py:4: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  cal_cars = cal_cars.groupby('읍면동_매핑')['합계_차량', '승  용', '승  합', '화  물', '특  수', '관  용', '자가용', '영업용',].sum()


In [21]:
cal_cars

,읍면동_매핑,합계_차량,승 용,승 합,화 물,특 수,관 용,자가용,영업용
0,구갈동,18625.0,16764.0,381.0,1427.0,53.0,1.0,17235.0,1389.0
1,구성동,17151.0,15401.0,379.0,1321.0,50.0,19.0,16553.0,579.0
2,기흥동,11102.0,9417.0,391.0,1262.0,32.0,0.0,10626.0,476.0
3,남사읍,13772.0,11156.0,352.0,2208.0,56.0,28.0,13490.0,254.0
4,동부동,7137.0,5573.0,254.0,1233.0,77.0,1.0,6859.0,277.0
5,동천동,22280.0,20322.0,308.0,1603.0,47.0,1.0,21512.0,767.0
6,마북동,13349.0,12454.0,229.0,645.0,21.0,5.0,13124.0,220.0
7,모현읍,16486.0,12192.0,720.0,3507.0,67.0,0.0,15616.0,870.0
8,백암면,6491.0,3607.0,255.0,2467.0,162.0,1.0,5225.0,1265.0
9,보라동,15267.0,13239.0,414.0,1542.0,72.0,0.0,14638.0,629.0


In [22]:
# 숫자 동은 다소 까다로워 엑셀로 처리
cal_cars2 = pd.read_csv("C:/Users/bhg\\Desktop/2022빅콘/sep_dong.csv",encoding='cp949')
cal_cars2.dropna(inplace=True)
final_agg = pd.concat([cal_cars,cal_cars2],ignore_index=True)
final_agg = final_agg.reset_index().drop(['index'],axis=1)
final_agg

,읍면동_매핑,합계_차량,승 용,승 합,화 물,특 수,관 용,자가용,영업용
0,구갈동,18625.0,16764.0,381.0,1427.0,53.0,1.0,17235.0,1389.0
1,구성동,17151.0,15401.0,379.0,1321.0,50.0,19.0,16553.0,579.0
2,기흥동,11102.0,9417.0,391.0,1262.0,32.0,0.0,10626.0,476.0
3,남사읍,13772.0,11156.0,352.0,2208.0,56.0,28.0,13490.0,254.0
4,동부동,7137.0,5573.0,254.0,1233.0,77.0,1.0,6859.0,277.0
5,동천동,22280.0,20322.0,308.0,1603.0,47.0,1.0,21512.0,767.0
6,마북동,13349.0,12454.0,229.0,645.0,21.0,5.0,13124.0,220.0
7,모현읍,16486.0,12192.0,720.0,3507.0,67.0,0.0,15616.0,870.0
8,백암면,6491.0,3607.0,255.0,2467.0,162.0,1.0,5225.0,1265.0
9,보라동,15267.0,13239.0,414.0,1542.0,72.0,0.0,14638.0,629.0


In [23]:
grid_countcust

,cell_id,cell_xcrd,cell_ycrd,읍면동_격자,tot_count_cust
0,84919076,127.109460,37.270844,구갈동,923.508
1,84935076,127.110020,37.271297,구갈동,3.048
2,84935081,127.110010,37.273550,구갈동,1262.039
3,84935082,127.110010,37.274000,구갈동,275.330
4,84935083,127.110010,37.274452,구갈동,181.521
...,...,...,...,...,...
18542,84647202,127.099620,37.319935,풍덕천2동,157.781
18543,84663200,127.100180,37.319490,풍덕천2동,2775.626
18544,84679200,127.100746,37.319940,풍덕천2동,8.130
18545,84695200,127.101300,37.320393,풍덕천2동,10.038


In [24]:
grids_agg = pd.merge(final_agg,grid_countcust[['cell_id', 'cell_xcrd', 'cell_ycrd', 'tot_count_cust', '읍면동_격자',]],left_on='읍면동_매핑',right_on='읍면동_격자').drop('읍면동_매핑',axis=1)
grids_agg.to_csv('grids_cust&car.csv')
grids_agg

,합계_차량,승 용,승 합,화 물,특 수,관 용,자가용,영업용,cell_id,cell_xcrd,cell_ycrd,tot_count_cust,읍면동_격자
0,18625.0,16764.0,381.0,1427.0,53.0,1.0,17235.0,1389.0,84919076,127.109460,37.270844,923.508,구갈동
1,18625.0,16764.0,381.0,1427.0,53.0,1.0,17235.0,1389.0,84935076,127.110020,37.271297,3.048,구갈동
2,18625.0,16764.0,381.0,1427.0,53.0,1.0,17235.0,1389.0,84935081,127.110010,37.273550,1262.039,구갈동
3,18625.0,16764.0,381.0,1427.0,53.0,1.0,17235.0,1389.0,84935082,127.110010,37.274000,275.330,구갈동
4,18625.0,16764.0,381.0,1427.0,53.0,1.0,17235.0,1389.0,84935083,127.110010,37.274452,181.521,구갈동
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18542,13064.0,12348.0,202.0,495.0,19.0,1.0,12947.0,116.0,84471195,127.093450,37.311802,5026.583,상현2동
18543,13064.0,12348.0,202.0,495.0,19.0,1.0,12947.0,116.0,84471197,127.093445,37.312702,763.386,상현2동
18544,13064.0,12348.0,202.0,495.0,19.0,1.0,12947.0,116.0,84471198,127.093445,37.313156,280.883,상현2동
18545,13064.0,12348.0,202.0,495.0,19.0,1.0,12947.0,116.0,84487194,127.094020,37.311806,13.628,상현2동


#### 100x100 확인

In [71]:
grids100 = pd.read_csv("C:/Users/bhg/Desktop/2022빅콘/100x100격자_지가차량cust.csv")
grids100.dropna(subset=['tot_count_'],inplace=True) #수요 없는 곳 드랍
grids100

C:\Users\bhg\AppData\Local\Temp\ipykernel_7280\1813529508.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  grids100 = pd.read_csv("C:/Users/bhg/Desktop/2022빅콘/100x100격자_지가차량cust.csv")


,val,합계_차,승 용,승 합,화 물,특 수,관 용,자가용,영업용,tot_count_,읍면동_,x,y
0,673171.08,18625.0,16764.0,381.0,1427.0,53.0,1.0,17235.0,1389.0,923.508,구갈동,127.109181,37.270617
1,523334.37,18625.0,16764.0,381.0,1427.0,53.0,1.0,17235.0,1389.0,3.048,구갈동,127.110304,37.271522
2,3227289.23,18625.0,16764.0,381.0,1427.0,53.0,1.0,17235.0,1389.0,1262.039,구갈동,127.110295,37.273325
3,2495153.56,18625.0,16764.0,381.0,1427.0,53.0,1.0,17235.0,1389.0,275.330,구갈동,127.110290,37.274226
4,2495153.56,18625.0,16764.0,381.0,1427.0,53.0,1.0,17235.0,1389.0,181.521,구갈동,127.110290,37.274226
...,...,...,...,...,...,...,...,...,...,...,...,...,...
18527,NaN,13064.0,12348.0,202.0,495.0,19.0,1.0,12947.0,116.0,5026.583,상현2동,127.093167,37.312027
18528,NaN,13064.0,12348.0,202.0,495.0,19.0,1.0,12947.0,116.0,763.386,상현2동,127.093162,37.312928
18529,NaN,13064.0,12348.0,202.0,495.0,19.0,1.0,12947.0,116.0,280.883,상현2동,127.093162,37.312928
18530,105462.79,13064.0,12348.0,202.0,495.0,19.0,1.0,12947.0,116.0,13.628,상현2동,127.094296,37.312031


In [81]:
tot_counts = grids100.groupby(['x','y']).aggregate({'val':np.mean,'합계_차':np.mean, '승  용':np.mean, '승  합': np.mean, '화  물' :np.mean, 
                                                   '특  수' : np.mean, '관  용' : np.mean, '자가용':np.mean, '영업용':np.mean, 'tot_count_':np.sum, '읍면동_':np.max})
tot_counts.reset_index().to_csv('100격자_전처리.csv',index=False)

### 후보지 불가 지역 산출

In [32]:
candidates = pd.read_csv("C:/Users/bhg/Desktop/2022빅콘/교차영역.csv")
candidates
# 불가지역이 교차한 격자는 31394개

,y,x,pnu,불가면적
0,407464.892,232054.184,4146135030110150000,137
1,407464.892,232054.184,4146135030110130000,10
2,407464.892,232054.184,4146135030110140000,235
3,407464.892,232054.184,4146135030110100000,53
4,426483.519,215146.735,4146125327107300016,194
...,...,...,...,...
169839,401473.527,215178.728,4146125928106940017,22
169840,401473.527,215178.728,4146125928106940018,4
169841,401473.527,215178.728,4146125928111260000,633
169842,401473.527,215178.728,4146125928107000004,18


In [33]:
candidates = (candidates.groupby(['x','y'])['불가면적'].sum()/100>=80).reset_index()
not_candidates = candidates.loc[candidates['불가면적']==True]
not_candidates
not_candidates.to_csv('불가지.csv',index=True)
# 971개 제외됨 